In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import copy
import pandas as pd
from models import CNN_Clasificator, LSTM__Clasificator
from get_embedded_data import get_data_glove_CNN, get_data_glove_LSTM, get_data_word2vec_CNN, get_data_word2vec_LSTM, split_data, MAPPING

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Python311\Lib\site-packages\torchtext\vocab\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
c:\Python311\Lib\site-packages\torchtext\utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warn

In [2]:
device = torch.device("cuda")
device

device(type='cuda')

In [3]:
def train_LSTM(max_epoch, network, train_dataloader, test_dataloader, optimizer, criterion, device):
    train_data_df = []
    network.train()
    best_network = network.state_dict()
    best_accuracy = 0
    dict_for_stat = {
        0: [0,0,0],
        1: [0,0,0],
        2: [0,0,0],
        3: [0,0,0],
        4: [0,0,0],
        5: [0,0,0],
        6: [0,0,0],
        7: [0,0,0],
        8: [0,0,0]
    }
    for epoch in range(max_epoch):
        running_loss = 0.0
        correct = 0
        total = 0
        network.train()
        for i, data in enumerate(train_dataloader, 0):
            inputs, labels, x_len = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            network.prep(200)
            outputs = network(inputs)

            new_preds = torch.empty((0,9)).to(device)
            for i in range(len(outputs)):
                new_preds = torch.vstack((new_preds, outputs[i][x_len[i]-1]))

            loss = criterion(new_preds, labels)
            loss.backward()
            optimizer.step()


            total += labels.size(0)
            _, predicted = torch.max(new_preds.data, 1)
            correct += (predicted == labels).sum().item()

            running_loss += loss.item()

        val_correct = 0
        val_total = 0
        network.eval()
        for i, data in enumerate(test_dataloader, 0):
            inputs, labels, x_len = data
            inputs, labels = inputs.to(device), labels.to(device)
            network.prep(1)
            outputs = network(inputs)
            new_preds = torch.empty((0,9)).to(device)
            for i in range(len(outputs)):
                new_preds = torch.vstack((new_preds, outputs[i][x_len[i]-1]))
            val_total += labels.size(0)
            _, predicted = torch.max(new_preds.data, 1)
            val_correct += (predicted == labels).sum().item()
        if 100 * val_correct / val_total > best_accuracy:
            best_accuracy = 100 * val_correct / val_total
            best_network = copy.deepcopy(network.state_dict())

        train_loss = running_loss / 2000
        train_accuracy = round(100 * correct / total, 3)
        val_accuracy = round(100 * val_correct / val_total, 3)
        train_data_df.append([epoch+1, max_epoch, round(train_loss, 3), train_accuracy, val_accuracy])
        print(f"Test accuracy: {100 * val_correct / val_total}")

        print('[%d/%d] loss: %.3f accuracy: %d' %
          (epoch+1, max_epoch, running_loss / 2000, 100 * correct / total))

        running_loss = 0.0

    network.load_state_dict(best_network)
    best_network = network

    correct = 0
    total = 0
    all_labels = []
    all_predictions = []
    best_network.eval()
    for i, data in enumerate(test_dataloader, 0):
        inputs, labels, x_len = data
        inputs, labels = inputs.to(device), labels.to(device)
        network.prep(1)
        outputs = best_network(inputs)
        new_preds = torch.empty((0,9)).to(device)
        for i in range(len(outputs)):
            new_preds = torch.vstack((new_preds, outputs[i][x_len[i]-1]))
        total += labels.size(0)
        _, predicted = torch.max(new_preds.data, 1)
        correct += (predicted == labels).sum().item()
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())
        for pr, lab in zip(predicted, labels):
            pr, lab = pr.item(), lab.item()
            if pr == lab:
                # TP
                dict_for_stat[pr][0] += 1
                continue
            # FN
            dict_for_stat[lab][1] += 1
            # FP
            dict_for_stat[pr][2] += 1

    pr_rec_f1 = {}
    for key in dict_for_stat.keys():
        tp, fn, fp = dict_for_stat[key]
        precision = -1 if tp+fp == 0 else tp/(tp+fp)
        recall = -1 if tp+fn == 0 else tp/(tp+fn)
        f1_score = -1 if tp+fn+fp == 0 else 2*tp/(2*tp+fn+fp)
        pr_rec_f1[key] = [precision, recall, f1_score]

    final_accuracy = round(100 * correct / total, 3)
    return best_network, final_accuracy, best_accuracy, all_labels, all_predictions, pr_rec_f1, pd.DataFrame(train_data_df, columns=["Epoch", "Max Epoch", "loss", "train data accuracy", "test data accuracy"])

In [4]:
def train_CNN(max_epoch, network, train_dataloader, test_dataloader, optimizer, criterion, device):
    train_data_df = []
    network.train()
    best_network = network.state_dict()
    best_accuracy = 0
    dict_for_stat = {
        0: [0,0,0],
        1: [0,0,0],
        2: [0,0,0],
        3: [0,0,0],
        4: [0,0,0],
        5: [0,0,0],
        6: [0,0,0],
        7: [0,0,0],
        8: [0,0,0]
    }
    for epoch in range(max_epoch):
        running_loss = 0.0
        correct = 0
        total = 0
        network.train()
        for i, data in enumerate(train_dataloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = network(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()


            total += labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()

            running_loss += loss.item()

        val_correct = 0
        val_total = 0
        network.eval()
        for i, data in enumerate(test_dataloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = network(inputs)
            val_total += labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            val_correct += (predicted == labels).sum().item()

        if 100 * val_correct / val_total > best_accuracy:
            best_accuracy = 100 * val_correct / val_total
            best_network = copy.deepcopy(network)

        train_loss = running_loss / 2000
        train_accuracy = round(100 * correct / total, 3)
        val_accuracy = round(100 * val_correct / val_total, 3)
        train_data_df.append([epoch+1, max_epoch, round(train_loss, 3), train_accuracy, val_accuracy])
        running_loss = 0.0

        print(f"Test accuracy: {100 * val_correct / val_total}")

        print('[%d/%d] loss: %.3f accuracy: %d' %
          (epoch+1, max_epoch, running_loss / 2000, 100 * correct / total))

    correct = 0
    total = 0
    all_labels = []
    all_predictions = []
    best_network.eval()
    for i, data in enumerate(test_dataloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = best_network(inputs)
        total += labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum().item()
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())
        for pr, lab in zip(predicted, labels):
            pr, lab = pr.item(), lab.item()
            if pr == lab:
                # TP
                dict_for_stat[pr][0] += 1
                continue
            # FN
            dict_for_stat[lab][1] += 1
            # FP
            dict_for_stat[pr][2] += 1

    pr_rec_f1 = {}
    for key in dict_for_stat.keys():
        tp, fn, fp = dict_for_stat[key]
        precision = -1 if tp+fp == 0 else tp/(tp+fp)
        recall = -1 if tp+fn == 0 else tp/(tp+fn)
        f1_score = -1 if tp+fn+fp == 0 else 2*tp/(2*tp+fn+fp)
        pr_rec_f1[key] = [precision, recall, f1_score]

    final_accuracy = round(100 * correct / total, 3)
    return best_network, final_accuracy, best_accuracy, all_labels, all_predictions, pr_rec_f1, pd.DataFrame(train_data_df, columns=["Epoch", "Max Epoch", "loss", "train data accuracy", "test data accuracy"])

In [5]:
models_list = ["CNN", "LSTM"]
embeddings_list = ['glove', 'word2vec']
labels_to_delete_list = [[], ['spinoza', "hegel", "plato"]]
weigth_list = [None, torch.tensor([1,1,2,3,1.33,1.33,5,1,2.5],dtype=torch.double).to(device)]

full_data = []
for labels_to_delete in labels_to_delete_list:
    X_train, X_test, y_train, y_test = split_data("data_set.csv", "author", "quote", test_size=0.2, separator="@", mapping=MAPPING,
                                                labels_to_delete=labels_to_delete)
    for name in models_list:
        for emb_name in embeddings_list:
            exec(f'train_dataloader = get_data_{emb_name}_{name}(200, X_train, y_train)')
            exec(f'test_dataloader = get_data_{emb_name}_{name}(1, X_test, y_test)')
            for weigths in weigth_list:
                if name == "CNN":
                    network = CNN_Clasificator().to(device)
                else: 
                    network = LSTM__Clasificator(90).to(device)
                criterion = nn.CrossEntropyLoss(weight=weigths)
                optimizer = optim.Adam(network.parameters())
                exec(f'best_network, test_acc, best_accuracy, all_labels, all_predictions, pr_rec_f1, train_data = train_{name}(max_epoch=70, network=network,\
                    train_dataloader=train_dataloader, test_dataloader=test_dataloader, optimizer=optimizer, criterion=criterion, device=device)')
                weigths = [] if weigths is None else [1,1,2,3,1.33,1.33,5,1,2.5]
                torch.save(best_network, f"train_models/{name}_{emb_name}_{labels_to_delete}_{weigths}")
                train_data.to_csv(f"train_data/{name}_{emb_name}_{labels_to_delete}_{weigths}.csv")
                full_data.append([f'{name}_{emb_name}', labels_to_delete, weigths, test_acc, best_accuracy, all_labels, all_predictions, pr_rec_f1])
full_data_df = pd.DataFrame(full_data, columns=["name", "labels_to_delete", "weigths", "test_acc", "best_accuracy", "all_labels", "all_predictions", "pr_rec_f1"])
full_data_df.to_csv("train_data/full_data.csv")

['aristotle' 'freud' 'hegel' 'kant' 'nietzsche' 'plato' 'sartre'
 'schopenhauer' 'spinoza']


c:\Python311\Lib\site-packages\torch\nn\modules\conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,


Test accuracy: 20.242914979757085
[1/70] loss: 0.000 accuracy: 16
Test accuracy: 20.242914979757085
[2/70] loss: 0.000 accuracy: 18
Test accuracy: 21.25506072874494
[3/70] loss: 0.000 accuracy: 18
Test accuracy: 21.05263157894737
[4/70] loss: 0.000 accuracy: 21
Test accuracy: 21.45748987854251
[5/70] loss: 0.000 accuracy: 22
Test accuracy: 24.291497975708502
[6/70] loss: 0.000 accuracy: 24
Test accuracy: 25.708502024291498
[7/70] loss: 0.000 accuracy: 25
Test accuracy: 26.11336032388664
[8/70] loss: 0.000 accuracy: 26
Test accuracy: 25.506072874493928
[9/70] loss: 0.000 accuracy: 29
Test accuracy: 27.125506072874494
[10/70] loss: 0.000 accuracy: 30
Test accuracy: 28.34008097165992
[11/70] loss: 0.000 accuracy: 31
Test accuracy: 24.696356275303643
[12/70] loss: 0.000 accuracy: 34
Test accuracy: 27.93522267206478
[13/70] loss: 0.000 accuracy: 35
Test accuracy: 30.161943319838056
[14/70] loss: 0.000 accuracy: 37
Test accuracy: 29.352226720647774
[15/70] loss: 0.000 accuracy: 36
Test accur

c:\Users\bgale\Desktop\Nowy folder (3)\NLP\get_embedded_data.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(xx_pad, dtype=torch.double), torch.tensor(yy), x_lens


Test accuracy: 16.194331983805668
[1/70] loss: 0.010 accuracy: 15
Test accuracy: 16.194331983805668
[2/70] loss: 0.010 accuracy: 16
Test accuracy: 17.611336032388664
[3/70] loss: 0.009 accuracy: 17
Test accuracy: 17.004048582995953
[4/70] loss: 0.009 accuracy: 17
Test accuracy: 17.408906882591094
[5/70] loss: 0.009 accuracy: 17
Test accuracy: 19.23076923076923
[6/70] loss: 0.009 accuracy: 19
Test accuracy: 22.469635627530366
[7/70] loss: 0.009 accuracy: 21
Test accuracy: 23.481781376518217
[8/70] loss: 0.009 accuracy: 21
Test accuracy: 18.016194331983804
[9/70] loss: 0.009 accuracy: 22
Test accuracy: 23.68421052631579
[10/70] loss: 0.009 accuracy: 21
Test accuracy: 21.25506072874494
[11/70] loss: 0.009 accuracy: 24
Test accuracy: 27.732793522267208
[12/70] loss: 0.009 accuracy: 26
Test accuracy: 24.898785425101213
[13/70] loss: 0.009 accuracy: 25
Test accuracy: 28.13765182186235
[14/70] loss: 0.009 accuracy: 25
Test accuracy: 26.720647773279353
[15/70] loss: 0.009 accuracy: 25
Test acc

...

Test accuracy: 16.59919028340081
[1/70] loss: 0.010 accuracy: 15
Test accuracy: 16.59919028340081
[2/70] loss: 0.009 accuracy: 15
Test accuracy: 17.206477732793523
[3/70] loss: 0.009 accuracy: 17
Test accuracy: 17.813765182186234
[4/70] loss: 0.009 accuracy: 18
Test accuracy: 20.040485829959515
[5/70] loss: 0.009 accuracy: 20
Test accuracy: 24.696356275303643
[6/70] loss: 0.009 accuracy: 22
Test accuracy: 24.898785425101213
[7/70] loss: 0.009 accuracy: 26
Test accuracy: 27.327935222672064
[8/70] loss: 0.009 accuracy: 31
Test accuracy: 28.94736842105263
[9/70] loss: 0.008 accuracy: 31
Test accuracy: 30.364372469635626
[10/70] loss: 0.008 accuracy: 33
Test accuracy: 31.57894736842105
[11/70] loss: 0.008 accuracy: 37
Test accuracy: 30.5668016194332
[12/70] loss: 0.008 accuracy: 39
Test accuracy: 32.18623481781376
[13/70] loss: 0.007 accuracy: 41
Test accuracy: 30.364372469635626
[14/70] loss: 0.007 accuracy: 43
Test accuracy: 30.97165991902834
[15/70] loss: 0.007 accuracy: 44
Test accurac

c:\Users\bgale\Desktop\Nowy folder (3)\NLP\get_embedded_data.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(xx_pad, dtype=torch.double), torch.tensor(yy), x_lens


Test accuracy: 18.52731591448931
[1/70] loss: 0.008 accuracy: 18
Test accuracy: 18.52731591448931
[2/70] loss: 0.008 accuracy: 19
Test accuracy: 19.477434679334916
[3/70] loss: 0.008 accuracy: 18
Test accuracy: 18.764845605700714
[4/70] loss: 0.007 accuracy: 17
Test accuracy: 19.71496437054632
[5/70] loss: 0.007 accuracy: 17
Test accuracy: 19.002375296912113
[6/70] loss: 0.007 accuracy: 19
Test accuracy: 21.140142517814727
[7/70] loss: 0.007 accuracy: 19
Test accuracy: 28.26603325415677
[8/70] loss: 0.007 accuracy: 20
Test accuracy: 21.61520190023753
[9/70] loss: 0.007 accuracy: 20
Test accuracy: 22.090261282660332
[10/70] loss: 0.007 accuracy: 19
Test accuracy: 27.315914489311165
[11/70] loss: 0.007 accuracy: 21
Test accuracy: 28.26603325415677
[12/70] loss: 0.007 accuracy: 23
Test accuracy: 29.216152019002376
[13/70] loss: 0.007 accuracy: 22
Test accuracy: 28.50356294536817
[14/70] loss: 0.007 accuracy: 24
Test accuracy: 28.978622327790973
[15/70] loss: 0.007 accuracy: 27
Test accura

c:\Users\bgale\Desktop\Nowy folder (3)\NLP\get_embedded_data.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(xx_pad, dtype=torch.double), torch.tensor(yy), x_lens


Test accuracy: 20.665083135391924
[1/70] loss: 0.009 accuracy: 16
Test accuracy: 19.477434679334916
[2/70] loss: 0.008 accuracy: 19
Test accuracy: 19.002375296912113
[3/70] loss: 0.008 accuracy: 20
Test accuracy: 23.27790973871734
[4/70] loss: 0.007 accuracy: 18
Test accuracy: 19.95249406175772
[5/70] loss: 0.007 accuracy: 18
Test accuracy: 26.60332541567696
[6/70] loss: 0.007 accuracy: 22
Test accuracy: 29.69121140142518
[7/70] loss: 0.007 accuracy: 23
Test accuracy: 27.315914489311165
[8/70] loss: 0.007 accuracy: 25
Test accuracy: 30.878859857482187
[9/70] loss: 0.007 accuracy: 25
Test accuracy: 31.116389548693586
[10/70] loss: 0.007 accuracy: 28
Test accuracy: 30.40380047505938
[11/70] loss: 0.007 accuracy: 26
Test accuracy: 31.59144893111639
[12/70] loss: 0.007 accuracy: 29
Test accuracy: 33.966745843230406
[13/70] loss: 0.007 accuracy: 31
Test accuracy: 35.15439429928741
[14/70] loss: 0.006 accuracy: 33
Test accuracy: 34.91686460807601
[15/70] loss: 0.006 accuracy: 34
Test accurac